In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image, ImageDraw
# import keras
import matplotlib.pyplot as plt

#### load photo convert into gray levels

In [ ]:
filename="Sample4"
img=Image.open(filename+'.jpg')
print(img)

gray = img.convert('L')  # making it gray
# gray.save(filename+"_gray.jpg")

#### crop

In [ ]:
print("original size:",gray.size)
xc=int(gray.size[0]/2)
yc=int(gray.size[1]/2)
hw=min(xc,yc)
gray=gray.crop((xc-hw, yc-hw, xc+hw, yc+hw))
print("cropped size:", gray.size)

#### resize

In [ ]:
size=(1024,1024)
gray.thumbnail(size)
# print("resized to:", gray.size)
# gray.save(filename+'_small.jpg')

#### normalize image

In [ ]:
npimage = np.array(gray)
print('min:',npimage.min(),'max:',npimage.max(),'mean:',npimage.mean())

def normalize(x,xmin,xmax):
    x=x-xmin
    return x/(xmax-xmin)*255

def bw(x):
    return np.where(x < 100, 0, 1)

data=bw(normalize(npimage,npimage.min(),npimage.max()))

#### remove "small" features

In [ ]:
mask_data=data.copy()
current_figure_size=0
edge_points=[]
sur=[(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]

def resolve_points():
    global current_figure_size
    (x,y)=edge_points.pop()
    mask_data[x,y]=200
    current_figure_size+=1
    x=min(x,1022)
    x=max(x,1)
    y=min(y,1022)
    
    for s in sur:
        cx=x+s[0]
        cy=y+s[1]
        if mask_data[cx,cy]==0:
            mask_data[cx,cy]=200
            edge_points.append((cx,cy))
    
    return len(edge_points)
            
for i in range(1024):
#     print('row',i)
    for j in range(1024):
        if mask_data[i,j]==0:
            edge_points.append((i,j))
            while resolve_points():
                pass
                
            print("figure size:", current_figure_size)
            if current_figure_size<150:
                print("skipping")
                mask_data = np.where(mask_data == 200, 1, mask_data)
            else:
                mask_data = np.where(mask_data == 200, 250, mask_data)
            current_figure_size=0
    
data=np.where(mask_data==250,0,1)

#### save final image

In [ ]:
# im = Image.fromarray(np.uint8(data*254))
# im.save(filename+'_small_BW.jpg')

#### move 45 degree line starting from corners. Stop when sum of black pixels>5.

In [ ]:
ndata=np.where(data == 1, 0, 1)

def get_corner(cor):
    for i in range(400):
        sum=0
        for j in range(i):
            x=i-j
            y=j
            if cor==3:
                x=1023-(i-j)
            elif cor==2:
                x=1023-(i-j)
                y=1023-j
            elif cor==1:
                y=1023-j
            sum+=ndata[x,y]
        if sum>4:
#             print(i, sum)
            pos=[]
            for j in range(i):
                x=i-j
                y=j
                if cor==3:
                    x=1023-(i-j)
                elif cor==2:
                    x=1023-(i-j)
                    y=1023-j
                elif cor==1:
                    y=1023-j
                if ndata[x,y]: pos.append((y,x))
#             print(pos)
            return pos[int(len(pos)/2)]
corners=[]        
for c in range(4):
    corners.append(get_corner(c))
print(corners)
corners.append(corners[0])

In [ ]:
# draw = ImageDraw.Draw(im)
# draw.line(corners, fill=128, width=3)
# draw.text(corners[0],'ZERO')
# draw.text(corners[1],'ONE')
# draw.text(corners[2],'TWO')
# draw.text(corners[3],'THREE')
# im.save(filename+'_small_BW_borders.jpg')

#### remap

In [ ]:
rsize = 900
remapped = np.zeros((rsize, rsize))

def interp(p1, p2, p):
    return (p1[0]+(p2[0]-p1[0])*p, p1[1]+(p2[1]-p1[1])*p)

for x in range(rsize):
    for y in range(rsize):
        xp=x/rsize
        yp=y/rsize
        p1=interp(corners[0],corners[1],xp)
        p2=interp(corners[3],corners[2],xp)
        p3=interp(corners[1],corners[2],yp)
        p4=interp(corners[0],corners[3],yp)
        xmu=(p1[0]*p2[1]-p1[1]*p2[0])*(p3[0]-p4[0])-(p1[0]-p2[0])*(p3[0]*p4[1]-p3[1]*p4[0])
        ymu=(p1[0]*p2[1]-p1[1]*p2[0])*(p3[1]-p4[1])-(p1[1]-p2[1])*(p3[0]*p4[1]-p3[1]*p4[0])
        md=(p1[0]-p2[0])*(p3[1]-p4[1])-(p1[1]-p2[1])*(p3[0]-p4[0])
        xm=int(xmu/md)
        ym=int(ymu/md)
#         if x==0: 
#             print( 'xp:',xp,'yp:',yp, p1,p2,p3,p4)
#             print( 'xm:',xm,'ym:',ym)
        remapped[x,rsize-1-y]=data[ym,xm]
    
im = Image.fromarray(np.uint8(remapped*254))
im.save(filename+'_fixed.bmp')

#### cut out fields

In [ ]:
fields=[]
pad=10
for i in range(9):
    for j in range(9):
        fi=remapped[i*100+pad:(i+1)*100-pad,j*100+pad:(j+1)*100-pad]
#         im = Image.fromarray(np.uint8(fi*254))
#         im.save(filename+'_'+str(i)+'_'+str(j)+'_field.jpg')
        fields.append(fi)
